## Background

- You are an analyst with a company manufacturing electric vehicles.
- You have multiple suppliers providing multiple parts.
- Executives at your company have asked you to analyze inventory management.
- **Key Question**: Can the company reduce costs by using a single supplier instead of three suppliers?

---

## Project Tasks

- **Describe** the structure of order data provided by the inventory management team.
- **Clean** the dataset:
  - Remove duplicate rows.
  - Remove rows with missing data.
- **Compute** the average time to delivery by supplier:
  - Delivery time = `expected delivery date` − `order date`.
- **Create** an inventory management simulation:
  - Randomly select demand from a **normal distribution**:
    - Mean: 50
    - Standard deviation: 10
  - Holding cost: **$10 per unit per day**.

  - Shortage cost: **$50 per unit per day**.

---

## Other Project Tasks

- For each supplier, use their **average days to delivery** as the order lead time.
- Run a **180-day simulation** for each supplier and calculate inventory management costs.
- **Goal**: Identify which supplier results in the **lowest costs**.

In [11]:
import numpy as np
import pandas as pd
import random as rd
import matplotlib.pyplot as plt

data = pd.read_csv('B2B_Order_Data.csv')
# Clean data
data = data.drop_duplicates()
data = data.dropna()
data.info()
# Convert to datetime
data['Expected_Delivery_Date'] = pd.to_datetime(data['Expected_Delivery_Date'])
data['Order_Date'] = pd.to_datetime(data['Order_Date'])
# Compute delivery time in days
data['Delivery_Time'] = (data['Expected_Delivery_Date'] - data['Order_Date']).dt.days


<class 'pandas.core.frame.DataFrame'>
Index: 996 entries, 0 to 1001
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Supplier_ID             996 non-null    object 
 1   Order_Date              996 non-null    object 
 2   Shipping_Date           996 non-null    object 
 3   Expected_Delivery_Date  996 non-null    object 
 4   Product_Code            996 non-null    object 
 5   Product_Name            996 non-null    object 
 6   Per_Unit_Cost           996 non-null    float64
 7   Quantity                996 non-null    int64  
 8   Total_Cost              996 non-null    float64
dtypes: float64(2), int64(1), object(6)
memory usage: 77.8+ KB


In [43]:
'''
SIMULATION PARAMS
'''
demand_mean = 50
demand_stddev = 10

num_simulations = 100
sim_days = 180
holding_cost = 10  # Per unit per day
shortage_cost = 50  # Per unit per day

order_lead_time_sup1 = data.loc[data['Supplier_ID'] == 'SUP1', 'Delivery_Time'].mean()
order_lead_time_sup2 = data.loc[data['Supplier_ID'] == 'SUP2', 'Delivery_Time'].mean()
order_lead_time_sup3 = data.loc[data['Supplier_ID'] == 'SUP3', 'Delivery_Time'].mean()
suppliers_dt = [order_lead_time_sup1, order_lead_time_sup2, order_lead_time_sup3]


def run_sim(order_lead_time):
    total_holding_cost = 0
    total_shortage_cost = 0
    inventory_level = 0
    daily_demand = np.random.normal(demand_mean, demand_stddev, sim_days)

    for day in range(sim_days):
        demand = daily_demand[day]
        inventory_level -= demand
        # Check for new arriving orders
        if day % order_lead_time == 0:
            inventory_level += demand_mean * order_lead_time
        # Calculate shortage cost
        if inventory_level < demand:
            total_shortage_cost += abs(inventory_level) * shortage_cost
            inventory_level = 0
        # Calculate holding cost
        total_holding_cost += inventory_level * holding_cost

    return total_holding_cost, total_shortage_cost


results = {'Supplier': [], 'Simulation': [], 'Total_Holding_Cost': [], 'Total_Shortage_Cost': []}
for index, supplier in enumerate(suppliers_dt):
    supplier_cost = 0
    supplier_cost_list = []
    for simulation in range(num_simulations):
        total_holding_cost, total_shortage_cost = run_sim(supplier)
        sim_cost = total_holding_cost + total_shortage_cost
        supplier_cost += sim_cost
        results['Supplier'].append(index + 1)
        results['Simulation'].append(simulation + 1)
        results['Total_Holding_Cost'].append(total_holding_cost)
        results['Total_Shortage_Cost'].append(total_shortage_cost)
        supplier_cost_list.append(sim_cost)
    print(
        f'Supplier {index + 1} total sims cost: {supplier_cost}\n Supplier mean cost: {np.array(supplier_cost_list).mean()}')

results_df = pd.DataFrame(results)


Supplier 1 total sims cost: 46166452.939279385
 Supplier mean cost: 461664.52939279395
Supplier 2 total sims cost: 45822079.45314386
 Supplier mean cost: 458220.7945314385
Supplier 3 total sims cost: 46139186.691365846
 Supplier mean cost: 461391.8669136584
